<a href="https://colab.research.google.com/github/Waidhoferj/CSC-566-Project/blob/frame-loader/FacialLandmarks.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Facial Landmarks
Experimentation with facial landmarks models and datasets.

In [1]:
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.patches import Circle
import pandas as pd
import tensorflow as tf
from tensorflow.keras.models import Model, Sequential
from tensorflow.keras import layers
from google.colab import drive
import os
from scipy.io import loadmat
import random
!pip install mtcnn
from mtcnn import MTCNN
import cv2

In [2]:
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


Add your name and filepath to the project folder so that you can load the datasets.


In [3]:
USER = "John"
USER_FILEPATHS = {
    "John" : "/content/drive/MyDrive/CSC 566 Project",
    "Jeremy" : "/content/drive/MyDrive/School/Undergrad/2021 Spring/CSC 566/CSC 566 Project",
    "Ty" : "/content/drive/MyDrive/CSC 566 Project"
}
PROJECT_FILEPATH = USER_FILEPATHS[USER]

In [4]:
# Sanity check for loading data
def display_landmarks(img, points):
  """
  Displays portrait with landmark dots drawn on the face.
  Assumes that points are in (68,2)
  """
  #If we are reading from .mat files directly (2,68)
  if points.shape[0] == 2:
    points = points.transpose(1,0)
  #If reading from model output
  elif len(points.shape) == 1:
    points = points.reshape(-1,2)
  fig,ax = plt.subplots(1)
  ax.set_aspect('equal')
  ax.imshow(img)
  for p in points:
      circ = Circle(p)
      ax.add_patch(circ)
  plt.show()

#points = loadmat(AFW_DATASET + "/70037463_1.mat")["pt2d"]
#img  = plt.imread( AFW_DATASET+"/70037463_1.jpg")

#display_landmarks(img,points)



In [5]:
path = os.path.join(PROJECT_FILEPATH, "webCamData", "inputVideo.mp4")
vcap = cv2.VideoCapture(path)

success = 1
i = 0
frames = []
image_shape = (vcap.get(4), vcap.get(3), 3)

while success and i < 50:
  success, frame = vcap.read()
  frame = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
  frames.append(frame)
  i+=1

In [10]:

mtcnn = MTCNN()
cropped_frames = []
for i, frame in enumerate(frames):
  print(f"\r{i}",end="")
  faces = mtcnn.detect_faces(frame)
  if len(faces) > 0:
    x,y,w,h = faces[0]["box"]
    cropped_frames.append(frame[x:x+w, y:y+h])

99

In [11]:
def relu6(x):
  return tf.clip_by_value(x, 0, 6)

model_path = os.path.join(PROJECT_FILEPATH, "models", "PFLD.h5")
model = tf.keras.models.load_model(model_path, custom_objects={"relu6": relu6})

### Analyze Results

In [ ]:
input_data[0].shape

In [12]:
input_data = [tf.image.resize(frame, (450,450)).numpy().astype("uint8") for frame in cropped_frames]
norm_points, angles = model.predict(np.array(input_data))

In [ ]:
input_data[0].shape

In [ ]:
#@title View Predicted Images
image_index = 5 #@param {type:"slider", min:1, max:16, step:1}
points = norm_points * 450

i = 0
for frame in input_data:
  if (i == image_index):
    display_landmarks(frame, points[image_index])
    break
  i += 1